In [1]:
import scanpy as sc
import os
import pandas as pd
import numpy as np

import os, sys
from pathlib import Path

script_dir = Path().resolve()
repo_dir = script_dir
src_dir = repo_dir / 'src'
data_dir = repo_dir / 'data'
sys.path.append(str(src_dir))
sys.path.append(str(data_dir))

from dataset.dataloader import AnnDataDataset

PARAMETERS = {
    'hvgs': 20116,
    'num_genes': 20116,
    # 'hvgs': 5000,
    # 'num_genes': 5000,
    'latent_dimension': 50,
    'target_sum': 10000,
    'batch_size': 128,
    'num_epochs': 1,
}

In [2]:
data_path = data_dir / 'compressed' / 'tabula_muris'
tm_adata = sc.read(data_path / 'tabula_muris_filtered.h5ad')

In [3]:
tm_adata.obs.rename(columns={'cell_type': 'Celltype'}, inplace=True)


In [4]:
tm_adata.obs['Celltype'].replace(
    to_replace='pancreatic ductal cel',
    value='pancreatic ductal cell',
    inplace=True
)

/tmp/ipykernel_1375304/3173862297.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tm_adata.obs['Celltype'].replace(


In [5]:
from torch.utils.data import DataLoader

tm_dataset = AnnDataDataset(tm_adata)
tm_dataloader = DataLoader(tm_dataset, batch_size=2048, shuffle=True)

/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/

/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/

In [6]:
import pickle
with open(r'./data/pickled/tabula_muris/tm_dataset_jhu.pkl', 'wb') as f: # NOTE: 3,5 because apparently v3 already has both sexes
    pickle.dump(tm_dataset, f)

with open(r'./data/pickled/tabula_muris/tm_dataloader_jhu.pkl', 'wb') as f:
    pickle.dump(tm_dataloader, f)

with open(r'./data/pickled/tabula_muris/tm_adata_jhu', 'wb') as f:
    pickle.dump(tm_adata, f)

In [21]:
tm_adata.obs['tissue'].unique().tolist()

['tongue',
 'heart',
 'bone marrow',
 'mammary gland',
 'adipose tissue',
 'kidney',
 'liver',
 'lung',
 'limb muscle',
 'pancreas',
 'spleen',
 'thymus',
 'bladder lumen',
 'skin of body',
 'large intestine',
 'trachea']

In [33]:
import warnings
import torch
# Define functions to precompute data-dependent variables
def precompute_gene_clusters(dataset):
    most_significant_genes_dict = dataset.most_significant_genes_dict
    least_significant_genes_dict = dataset.least_significant_genes_dict
    gene_networks = dataset.gene_networks
    cell_type_categories = dataset.cell_type_categories
    code_to_celltype = dataset.code_to_celltype
    celltype_to_code = dataset.celltype_to_code
    gene_names = dataset.gene_names
    gene_name_to_index = dataset.gene_name_to_index
    index_to_gene_name = dataset.index_to_gene_name
    gene_dispersions = dataset.gene_dispersions
    print('Precomputed gene clusters!')
    return (most_significant_genes_dict, least_significant_genes_dict,
            gene_networks, gene_names, code_to_celltype, celltype_to_code,
            gene_name_to_index, index_to_gene_name, gene_dispersions)

def precompute_mu_sigma(dataloader, most_significant_genes_dict, least_significant_genes_dict, gene_name_to_index):
    all_expression_matrix = []
    cell_types_data = {}
    cell_types_msg_data = {}
    cell_types_lsg_data = {}
    for batch in dataloader:
        expression_matrix, cell_types = batch
        all_expression_matrix.append(expression_matrix)
        
        for cell_type in torch.unique(cell_types):
            cell_type = int(cell_type)
            cell_type_mask = cell_types == cell_type
            cell_type_expression_matrix = expression_matrix[cell_type_mask]
            # All genes
            if cell_type not in cell_types_data:
                cell_types_data[cell_type] = []
            cell_types_data[cell_type].append(cell_type_expression_matrix)

            # Most significant genes
            # msg_genes = most_significant_genes_dict[cell_type]
            # missing = [g for g in msg_genes if g not in gene_name_to_index]
            # print(f"{len(missing)} / {len(msg_genes)} genes missing:", missing[:10])

            # msg_gene_indices = [gene_name_to_index[g] for g in msg_genes]
            msg_genes = most_significant_genes_dict[cell_type]
            msg_gene_indices = [
                gene_name_to_index[g]          # lookup
                for g in msg_genes
                if g in gene_name_to_index     # keep only genes that exist
            ]
            if not msg_gene_indices:
                warnings.warn(f"No MSG genes found for cell type {cell_type}. "
                                "Check identifier mismatches.")

            msg_significant_gene_matrix = cell_type_expression_matrix[:, msg_gene_indices]
            if cell_type not in cell_types_msg_data:
                cell_types_msg_data[cell_type] = []
            cell_types_msg_data[cell_type].append(msg_significant_gene_matrix)
            
            # Least significant genes
            # lsg_gene_indices = [gene_name_to_index[g] for g in lsg_genes]
            lsg_genes = least_significant_genes_dict[cell_type]
            lsg_gene_indices = [
                gene_name_to_index[g]          # lookup
                for g in lsg_genes
                if g in gene_name_to_index     # keep only genes that exist
            ]
            if not lsg_gene_indices:
                warnings.warn(f"No LSG genes found for cell type {cell_type}. "
                                "Check identifier mismatches.")

            lsg_significant_gene_matrix = cell_type_expression_matrix[:, lsg_gene_indices]
            if cell_type not in cell_types_lsg_data:
                cell_types_lsg_data[cell_type] = []
            cell_types_lsg_data[cell_type].append(lsg_significant_gene_matrix)

    cell_type_mu_sigma = {}
    cell_type_msg_mu_sigma = {}
    cell_type_lsg_mu_sigma = {}
    # All genes
    for cell_type, cell_type_expression_matrix in cell_types_data.items():
        data_tensor = torch.cat(cell_type_expression_matrix, dim=0)
        mu = torch.mean(data_tensor, dim=0)
        sigma = torch.std(data_tensor, dim=0, unbiased=False)
        sigma = torch.clamp(sigma, min=1e-8)
        cell_type_mu_sigma[int(cell_type)] = (mu, sigma)
    
    # Most significant genes
    for cell_type, matrices in cell_types_msg_data.items():
        data_tensor = torch.cat(matrices, dim=0)
        mu = torch.mean(data_tensor, dim=0)
        sigma = torch.std(data_tensor, dim=0, unbiased=False)
        sigma = torch.clamp(sigma, min=1e-8)
        dispersion = sigma**2 / mu
        cell_type_msg_mu_sigma[int(cell_type)] = (mu, sigma, dispersion)
    
    # Least significant genes
    for cell_type, matrices in cell_types_lsg_data.items():
        data_tensor = torch.cat(matrices, dim=0)
        mu = torch.mean(data_tensor, dim=0)
        sigma = torch.std(data_tensor, dim=0, unbiased=False)
        sigma = torch.clamp(sigma, min=1e-8)
        dispersion = sigma**2 / mu
        cell_type_lsg_mu_sigma[int(cell_type)] = (mu, sigma, dispersion)

    data_tensor = torch.cat(all_expression_matrix, dim=0)
    global_mu_sigma = (torch.mean(data_tensor, dim=0),
                       torch.std(data_tensor, dim=0, unbiased=False))

    return cell_type_mu_sigma, global_mu_sigma, cell_type_msg_mu_sigma, cell_type_lsg_mu_sigma

# Precompute data-dependent variables before model initialization
(most_significant_genes_dict, least_significant_genes_dict,
 gene_networks, gene_names, code_to_celltype, celltype_to_code,
 gene_name_to_index, index_to_gene_name, gene_dispersions) = precompute_gene_clusters(tm_dataset)

cell_type_mu_sigma, global_mu_sigma, cell_type_msg_mu_sigma, cell_type_lsg_mu_sigma = precompute_mu_sigma(
    tm_dataloader, most_significant_genes_dict, least_significant_genes_dict, gene_name_to_index)

Precomputed gene clusters!


/oscar/data/rsingh47/wli115/scContrast/src/dataset/dataloader.py:126: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_item = torch.tensor(self.y[idx], dtype=torch.long)  # Return integer-encoded cell type


/tmp/ipykernel_1375304/4169309578.py:51: UserWarning: No MSG genes found for cell type 109. Check identifier mismatches.
  warnings.warn(f"No MSG genes found for cell type {cell_type}. "
/tmp/ipykernel_1375304/4169309578.py:51: UserWarning: No MSG genes found for cell type 4. Check identifier mismatches.
  warnings.warn(f"No MSG genes found for cell type {cell_type}. "
/tmp/ipykernel_1375304/4169309578.py:51: UserWarning: No MSG genes found for cell type 104. Check identifier mismatches.
  warnings.warn(f"No MSG genes found for cell type {cell_type}. "
/tmp/ipykernel_1375304/4169309578.py:91: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  sigma = torch.std(data_tensor, dim=0, unbiased=False)


In [ ]:
precomputed_dir = data_dir / 'pickled' / 'tabula_muris' / 'precomputed'
precomputed_dir.mkdir(parents=True, exist_ok=True)

precomputed_gene_clusters_path =  precomputed_dir / 'tm_dataset_jhu_precomputed_gene_clusters.pkl'
with open(precomputed_gene_clusters_path, 'wb') as f:
    pickle.dump(
        {
            "most_significant_genes_dict": most_significant_genes_dict,
            "least_significant_genes_dict": least_significant_genes_dict,
            "gene_networks": gene_networks,
            "gene_names": gene_names,
            "code_to_celltype": code_to_celltype,
            "celltype_to_code": celltype_to_code,
            "gene_name_to_index": gene_name_to_index,
            "index_to_gene_name": index_to_gene_name,
            "gene_dispersions": gene_dispersions,
        },
        f,
    )

precomputed_mu_sigma_path = precomputed_dir / 'tm_dataset_jhu_precomputed_mu_sigma.pkl'
with open(precomputed_mu_sigma_path, "wb") as f:
    pickle.dump(
        {
            "cell_type_mu_sigma": cell_type_mu_sigma,
            "global_mu_sigma": global_mu_sigma,
            "cell_type_msg_mu_sigma": cell_type_msg_mu_sigma,
            "cell_type_lsg_mu_sigma": cell_type_lsg_mu_sigma,
        },
        f,
    )